# Tabular Model: sequential data, with context

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from mostlyai import engine

# set up workspace
ws = Path("ws-tabular-sequential")

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/baseball"
trn_ctx_df = pd.read_csv(f"{url}/players.csv.gz")  # context data
trn_tgt_df = pd.read_csv(f"{url}/batting.csv.gz")  # target data

# execute the engine steps
engine.split(                         # split data as PQT files for `trn` + `val` to `{ws}/OriginalData/(tgt|ctx)-data`
  workspace_dir=ws,
  tgt_data=trn_tgt_df,
  ctx_data=trn_ctx_df,
  tgt_context_key="players_id",
  ctx_primary_key="id",
  model_type="TABULAR",
)
engine.analyze(workspace_dir=ws)      # generate column-level statistics to `{ws}/ModelStore/(tgt|ctx)-data/stats.json`
engine.encode(workspace_dir=ws)       # encode training data to `{ws}/OriginalData/encoded-data`
engine.train(                         # train model and store to `{ws}/ModelStore/model-data`
    workspace_dir=ws,
    max_training_time=2,              # limit TRAIN to 2 minute for demo purposes
)
engine.generate(workspace_dir=ws)     # use model to generate synthetic samples to `{ws}/SyntheticData`

In [ ]:
# load synthetic data
syn_tgt_df = pd.read_parquet(ws / "SyntheticData")
syn_tgt_df.head(5)

### QUALITY ASSURANCE

#### sequence lengths

In [ ]:
trn_seq_lens = trn_tgt_df.groupby("players_id").size()
syn_seq_lens = syn_tgt_df.groupby("players_id").size()

In [ ]:
print("tgt: ", np.quantile(trn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))
print("syn: ", np.quantile(syn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))

#### coherence

In [ ]:
syn_avg_teams_per_player = syn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
trn_avg_teams_per_player = trn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
syn_avg_teams_per_player, trn_avg_teams_per_player